# Lista 05 - Regresão

>Yanna Torres Gonçalves
>
>Matrícula: 587299
>
>Mestrado em Ciências da Computação


## Parte 01: limpeza e pré-processamento dos dados

In [1]:
!pip install pandas scikit-learn fuzzywuzzy python-Levenshtein spacy nltk sentence_transformers emoji unidecode torch
!python -m spacy download pt_core_news_sm

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached python_levenshtein-0.27.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached spacy-3.8.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached Unidecode-1.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached levenshtein-0.27.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
  Using cached rapidfuzz-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.13-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
import pandas as pd
import numpy as np
import re

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
import spacy
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### a) Ler o dataset fakeTelegram.BR_2022.csv

Utilizamos o mesmo dataset da lista 01, acessando pelo mesmo link.

[https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv](https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv)
   

In [3]:
data_url = "https://media.githubusercontent.com/media/yanna-torres/CKP9011-ciencia-de-dados/refs/heads/lista-01/data/fakeTelegram.BR_2022.csv"

df = pd.read_csv(data_url)

In [4]:
df.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,557586.0,444157.000000,167238.000000,5.575860e+05
mean,5.0,0.017330,0.312245,4.450617e+05
std,0.0,0.464165,0.293699,4.860211e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078454,2.127500e+04
50%,5.0,0.000000,0.197577,1.210935e+05
75%,5.0,0.318200,0.490351,9.726045e+05
max,5.0,0.999200,1.000000,1.516436e+06


### b) Remova os trava-zaps, as linhas repetidas e textos com menos de 5 palavras

#### Removendo os trava-zaps

In [5]:
def has_trava_zap(message):
    if not isinstance(message, str):
        return False  # Garante que só analisa strings

    # Regra 1: muito longo
    if len(message) > 10000:
        return True

    # Regra 2: caracteres invisíveis
    invisible = ''.join(chr(c) for c in range(0x200B, 0x200F + 1)) + '\uFEFF'
    if sum(message.count(c) for c in invisible) > 100:
        return True

    # Regra 3: emojis repetidos
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF"
                              "\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)
    emojis = emoji_pattern.findall(message)
    if any(len(e) > 20 for e in emojis):
        return True

    # Regra 4: baixa diversidade
    unique = set(message)
    if len(unique) < 10 and len(message) > 500:
        return True

    return False

In [6]:
df['trava_zap'] = df['text_content_anonymous'].apply(has_trava_zap)

In [7]:
df_clean = df[df['trava_zap'] == False]

In [8]:
df_clean.drop(columns=['trava_zap'], inplace=True)

/tmp/ipykernel_242/1420576694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(columns=['trava_zap'], inplace=True)


In [9]:
df_clean.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,556305.0,442876.000000,166416.000000,5.563050e+05
mean,5.0,0.017676,0.311496,4.448514e+05
std,0.0,0.463547,0.293146,4.859544e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078334,2.125100e+04
50%,5.0,0.000000,0.196892,1.208690e+05
75%,5.0,0.318200,0.488192,9.724220e+05
max,5.0,0.999200,1.000000,1.516436e+06


#### Removendo as linhas repetidas

In [10]:
df_clean = df_clean.drop_duplicates()

In [11]:
df_clean.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,556305.0,442876.000000,166416.000000,5.563050e+05
mean,5.0,0.017676,0.311496,4.448514e+05
std,0.0,0.463547,0.293146,4.859544e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078334,2.125100e+04
50%,5.0,0.000000,0.196892,1.208690e+05
75%,5.0,0.318200,0.488192,9.724220e+05
max,5.0,0.999200,1.000000,1.516436e+06


#### Removendo textos com menos de 5 palavras

In [12]:
df_clean = df_clean[df_clean['text_content_anonymous'].str.split().str.len() >= 5]

In [13]:
df_clean.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,338045.0,337998.000000,166402.000000,3.380450e+05
mean,5.0,0.022497,0.311514,4.214875e+05
std,0.0,0.529214,0.293150,4.842346e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.381800,0.078340,1.988800e+04
50%,5.0,0.000000,0.196925,8.473500e+04
75%,5.0,0.505300,0.488286,9.726970e+05
max,5.0,0.999200,1.000000,1.516436e+06


### c) Agrupe as linhas com postagens iguais ou semelhantes

In [14]:
df_clean.columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'message_type',
       'messenger', 'media_name', 'media_md5'],
      dtype='object')

In [15]:
import emoji
import unidecode

def normalize_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = emoji.replace_emoji(text, "")
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = unidecode.unidecode(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [16]:
df_clean['text_normalized'] = df_clean['text_content_anonymous'].apply(normalize_text)

In [17]:
len(df_clean)

338045

In [18]:
df_clean = df_clean[df_clean['text_normalized'].notnull()]
df_clean = df_clean[df_clean['text_normalized'] != ""]

In [19]:
len(df_clean)

337529

In [20]:
# Remove exact duplicates and count occurrences as "shares"
df_grouped = (
    df_clean.groupby('text_normalized', as_index=False)
      .agg({
            'date_message': 'min',
            'id_member_anonymous': 'first',
            **{col: 'first' for col in df_clean.columns if col != 'text_normalized' and col != 'date_message'}
      })
)

In [21]:
share_counts = df_clean['text_content_anonymous'].value_counts().to_dict()
df_grouped['shares'] = df_grouped['text_content_anonymous'].map(share_counts)

In [22]:
len(df_grouped)

214429

In [23]:
df_grouped.head(5)

,text_normalized,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,shares
0,0 800nana alaia azzedineparis france011 331jea...,2022-10-02 02:53:42,None,6e4192de8ce3464c5263b0a937e53529,None,None,None,False,False,",,""0 800"",""N/A"",""N/A"",\n\n""Alaia, Azzedine"",""P...",5,2022-10-03 04:54:51.879906,0.0000,0.003349,19900,Texto,telegram,None,None,1
1,0 a 10 vai ter 2o turno o que vocs acham,2022-10-03 00:11:39,None,b11f2df64ac19aad47a50accf32052d6,None,None,None,False,False,"0 a 10 vai ter 2o turno, o que vocês acham ?",5,2022-10-03 05:18:48.986706,0.0000,NaN,154118,Texto,telegram,None,None,1
2,0 cabea de ovo no respeita mais nada eles e do...,2022-10-28 02:47:19,None,e56ec342fc599ebb4ed89655eb6f03aa,None,None,None,False,False,0 cabeça de ovo não respeita mais nada eles ...,5,2022-10-28 02:47:24.04081,0.0258,NaN,540263,Texto,telegram,None,None,1
3,0 g0lpe t a t3 tem ch0que derealidade all ads ...,2022-09-29 03:44:33,None,5b10d7739171149be6d9961e3350c071,3327c1bedc6235177f5d48885d87b39d.jpg,url,https://www.youtube.com/watch?v=gwYdiCsaBoM,True,True,'0 G0LPE TÁ AÍ! - T$3 TEM CH0QUE DEREALIDADE'\...,5,2022-10-03 04:07:17.895291,-0.6352,NaN,1052465,Url,telegram,None,3327c1bedc6235177f5d48885d87b39d,1
4,0 mrs jenniferbrooksfx,2022-10-18 16:11:18,32d1832a2df01fd4a2a59b3ef8642c58,5b10d7739171149be6d9961e3350c071,None,None,https://t.me/+8neVAtvN9_4xODM0,False,True,https://t.me/+8neVAtvN9_4xODM0\n𝐈 𝐡𝐚𝐯𝐞 𝐭𝐫𝐢𝐞𝐝 𝐬...,5,2022-10-18 16:11:32.552063,0.0000,0.111523,1147539,Texto,telegram,None,None,2


In [24]:
from sentence_transformers import SentenceTransformer
import torch

# Load a multilingual model (Portuguese-friendly) and move to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)

2025-06-08 20:50:30.938706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-08 20:50:30.938784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-08 20:50:30.940046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-08 20:50:30.947277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# Get embeddings (batch-processed on GPU)
texts = df_grouped['text_normalized'].tolist()
print("Encoding texts into embeddings...")
embeddings = model.encode(texts, batch_size=128, show_progress_bar=True, device=device)
df_grouped['embedding'] = embeddings.tolist()

Encoding texts into embeddings...


Batches:   0%|          | 0/1676 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
faiss.normalize_L2(embeddings)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

In [ ]:
similarity_threshold = 0.85
k = 10

clusters = defaultdict(list)
removed_indices = set()

for i in tqdm(range(len(df_grouped))):
    if i in removed_indices:
        continue
    vec = embeddings[i].reshape(1, -1)
    D, I = index.search(vec, k)
    for sim, j in zip(D[0][1:], I[0][1:]):
        if j == -1 or j == i or j in removed_indices:
            continue
        if sim >= similarity_threshold:
            clusters[i].append(j)
            removed_indices.add(j)

In [ ]:
results = []
used = set()

for base_idx, sim_indices in clusters.items():
    if base_idx in used:
        continue
    total_count = df_grouped.iloc[base_idx]['share_count']
    used.add(base_idx)
    for sim_idx in sim_indices:
        total_count += df_grouped.iloc[sim_idx]['share_count']
        used.add(sim_idx)
    row = df_grouped.iloc[base_idx].copy()
    row['share_count'] = total_count
    results.append(row)

# Add remaining unmatched
remaining = df_grouped.loc[~df_grouped.index.isin(used)].copy()
final_df = pd.concat([pd.DataFrame(results), remaining], ignore_index=True)

In [ ]:
len(final_df)

In [ ]:
final_df[['text_content_anonymous', 'share_count']].head()

---

### d) Colunas Auxiliares

Para facilitar a recuperação de dados em alguns casos, vamos criar colunas auxiliares.

In [ ]:
stop_words = set(stopwords.words('portuguese', 'english'))

def clean_text(text):
  if not isinstance(text, str):
    return ''
  text_no_punct = text.translate(str.maketrans('', '', string.punctuation))
  tokens = word_tokenize(text_no_punct)
  filtered = [word for word in tokens if word.lower() not in stop_words]
  return ' '.join(filtered)

df_clean['text_no_stopwords'] = df_clean['text_content_anonymous'].apply(clean_text)

In [ ]:
df_clean['word_count'] = df_clean['text_content_anonymous'].apply(lambda x: len(x.split()) if pd.notnull(x) else 0)
df_clean['character_count'] = df_clean['text_content_anonymous'].apply(lambda x: len(x) if pd.notnull(x) else 0)

In [ ]:
text_counts = df_clean['text_content_anonymous'].value_counts()
df_clean['viral'] = df_clean['text_content_anonymous'].apply(lambda x: True if pd.notnull(x) and text_counts[x] > 1 else False)

In [ ]:
def classify_misinformation(score):
    if score >= 0.66:
        return 'Misinformation'
    elif score >= 0.33:
        return 'Neutral'
    else:
        return 'Non-misinformation'

# Criar coluna de categoria
df_clean['misinformation_category'] = df_clean['score_misinformation'].apply(classify_misinformation)

In [ ]:
df_clean['sentiment'] = df_clean['score_sentiment'].apply(lambda x: "Positive" if x >= 0.05 else ("Negative" if x <= -0.05 else "Neutral"))

In [ ]:
df_clean.info()

In [ ]:
df_clean.describe()

## Parte 02: Modelo Preditivo

Utilizando os dados referente a postagens no Telegram, crie um modelo preditivo
(regressor) para, dado os dados de uma postagem, prever a quantidade de
compartilhamentos dessa mensagem, o que é denominado potencial de “viralização”.